# Messungen mit JupyterLab auf dem STEMlab

Wie Sie JupyterLab auf einem Red Pitaya STEMlab starten und ein neues Projekt erstellen, sehen Sie hier:

Der Code in den folgenden Zellen erzeugt das gleiche Signal wie in @fig-19-06, aber er zeichnet die Messdaten in einem
xy-Diagramm auf. Für die Messung der IV-Kurve der Diode ist die x-Achse (Abszisse) die Diodenspannung an IN2 und die
y-Achse (Ordinate) der Diodenstrom $I_D = (IN1-IN2)/R_1$.


In [ ]:
# Import libraries
from rp_overlay import overlay

import time
import numpy as np
import rp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d, LinearAxis, LabelSet, Label
from bokeh.plotting import figure, output_file, show
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

# Initialize fpga modules
fpga = overlay()
rp.rp_Init()

# Configure OUT1 generator channel
channel = rp.RP_CH_1
waveform = rp.RP_WAVEFORM_TRIANGLE
ampl = 0.5
offset = 0.5
freq = 2000

# R1 resistor value
R1 = 10

# Acquisition paramters
dec = rp.RP_DEC_1

trig_lvl = 0.5
trig_dly = 0

acq_trig_sour = rp.RP_TRIG_SRC_NOW
N = 16384

# Reset Signal Generator
rp.rp_GenReset()
rp.rp_AcqReset()

# Set Signal Generator
print("Gen_start")
rp.rp_GenWaveform(channel, waveform)
rp.rp_GenFreqDirect(channel, freq)
rp.rp_GenAmp(channel, ampl)

# Output Signal Generator
rp.rp_GenOutEnable(channel)
rp.rp_GenTriggerOnly(channel)

# Specify generator trigger source
rp.rp_GenTriggerSource(channel, gen_trig_sour)

# Enable output synchronisation
rp.rp_GenOutEnableSync(True)

# Acquisition 
# Set Decimation
rp.rp_AcqSetDecimation(dec)

# Set trigger level and delay
rp.rp_AcqSetTriggerLevel(rp.RP_T_CH_1, trig_lvl)
rp.rp_AcqSetTriggerDelay(trig_dly)


# Configure IN1 and IN2 oscilloscope input channels
for ch in osc:
    ch.filter_bypass = True

    # data rate decimation
    ch.decimation = 10

    # trigger timing [sample periods]
    N = ch.buffer_size
    ch.trigger_pre = 0
    ch.trigger_post = N

    # osc0 is controlling both channels
    ch.sync_src = fpga.sync_src["osc0"]
    ch.trig_src = fpga.trig_src["osc0"]

    # trigger level [V], edge ['neg', 'pos'] and holdoff time [sample periods]
    ch.level = 0.5
    ch.edg = 'pos'
    ch.holdoff = 0

    
# Initialize diode current and voltage
V = I = np.zeros(N)


# Plotting
hover = HoverTool(mode='vline', tooltips=[("V", "@x"), ("I", "@y")])
tools = "wheel_zoom, box_zoom, reset, pan"
p = figure(plot_height=500, plot_width=900,
           title="XY plot der Dioden IV-Kurve",
           toolbar_location="right",
           tools=(tools, hover))
p.xaxis.axis_label = 'Spannung in V'
p.yaxis.axis_label = 'Strom in mA'
r = p.line(V, I, line_width=1, line_alpha=0.7, color="blue")


# get and explicit handle to update the next show cell
target = show(p, notebook_handle=True)

Die nachfolgende Zelle ist eine Hauptschleife für die Erfassung und Neuaufnahme. Wenn Sie die Erfassung stoppen,
führen Sie einfach nur diese Zelle aus, um die Messungen erneut zu starten.

In [ ]:
# Messung von  I, V und plotten
while True:
    # Reset und Start
    osc[0].reset()
    osc[0].start()

    # Auf Daten warten
    while (osc[0].status_run()):
        pass

    V0 = osc[0].data(N-100)  # IN1 Signal
    V1 = osc[1].data(N-100)  # IN2 Signal
    I = ((V0-V1)/R1)*1E3     # 1E3 Umwandlung zu mA
    r.data_source.data['x'] = V0
    r.data_source.data['y'] = I

    push_notebook(handle=target)

# Stop Generator
rp.rp_Release()

Nach dem Ausführen des Codes sollten Sie die IV-Charakteristik der Diode erhalten.

In der @fig-19-08 ist die typische IV-Kennlinie einer Silizium-Diode dargestellt. Es ist ersichtlich, dass bei
Spannungen zwischen 0 V und 0.5 V der Diodenstrom nahe Null ist bis Spannungswerte nahe der Schwellenspannung von etwa
0.7 V erreicht werden. Ab dieser Spannung wird die Diode *eingeschaltet* (leitend) und der Diodenstrom $I_D$ wird nur
durch den Widerstand $R_1$ begrenzt. Falls bei abnehmender Diodenspannung die IV-Kurve nicht gleich ist, führt dies zur
*Diodenhysterese*.
Die obere Kurve aus @fig-19-08 zeigt, dass die untere Diodenspannung nach dem *Einschalten* der Diode einen höheren
Strom verursacht als bei vorangegangenen *Abschaltung* der Diode. Eine ideale Diode hat keine Hysterese, d.h. der
Diodenstrom ist unabhängig von früheren Diodenzuständen und nur abhängig von der Diodenspannung. 

